# Economic Drivers of Airbnb Pricing in Los Angeles: An Examination of Crime, Demographics, and Geography

## Libraries

In [1]:
# Uncomment the line below to install the required packages
# !pip -q install pandas numpy matplotlib scipy

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt